In [ ]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.patches import Patch
from matplotlib.transforms import Bbox
from matplotlib.lines import Line2D
from matplotlib.ticker import FormatStrFormatter
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable

from scipy.stats import gamma
from scipy.stats import norm
from scipy.stats import multivariate_normal
from scipy.stats import gaussian_kde
from scipy.stats import circvar
from scipy.special import expit
from scipy.special import logsumexp
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint
from scipy.signal import convolve
from scipy.interpolate import interp1d

import time
import pickle
import importlib
import datetime

import divebomb
import sktime
from sktime.transformations.panel.shapelets import ContractedShapeletTransform

In [ ]:
matplotlib.rcParams.update({'font.size': 12})

np.random.seed(0)
np.set_printoptions(suppress=True,precision=4)

# Import Data

In [ ]:
# import data
df = pd.read_csv('../../Dat/diary/kinematic_data_calibrated_I107_2020_50Hz.csv',
                parse_dates = [0], infer_datetime_format = True)

# Do some preliminary things
df = df.rename(columns={"p":"depth"})
df['elevation'] = -df['depth']

# make a backup
df_backup = df.copy()

In [ ]:
df = df_backup

In [ ]:
# plot data
fig = df.iloc[::1000].plot(x='Time',y='depth')
fig.invert_yaxis()
plt.show(fig)

In [ ]:
# record labels
forage_events = pd.to_datetime(['2020-08-25 12:33:22',
                                '2020-08-25 13:24:15',
                                '2020-08-25 16:59:06',
                                '2020-08-25 17:07:48',
                                '2020-08-25 17:40:50'])

# Add a couple of varaibles

In [ ]:
df[['d_roll','d_head','d_pitch']] = np.abs(((df[['roll','head','pitch']]\
                                             .rolling(11,center=True)\
                                             .mean()\
                                             .diff(11) + np.pi) % (2*np.pi)) - np.pi)

df['VeDBA'] = np.sqrt(df['Aw_1']**2 + df['Aw_2']**2 + df['Aw_3']**2)
df['jerk'] = np.abs(((df['VeDBA'].rolling(5,center=True).mean().diff(5))))

# Divide into Segments

In [ ]:
### Take the last minute (at depth) of each dive deeper than 20 meters

# set parameters
surface_thresh = 20
dive_duration_thresh = datetime.timedelta(minutes=1.5)
between_dive_thresh = datetime.timedelta(seconds=2)
h = 4500

# add dive numbers
df_at_depth = df[df['depth'] >= surface_thresh]
df_at_depth['new_dive'] = (df_at_depth['Time'].diff() > between_dive_thresh)
df_at_depth['divenum'] = df_at_depth['new_dive'].cumsum()

In [ ]:
# get each dive
gbo = df_at_depth.groupby('divenum')

# get times
time = []

# get shapelet candidates
depth = []
roll = []

# get scalar features
htv = []
hv = []
jp = []

# get labels
labels = []

for divenum,group_df in gbo:
    
    # get the dataframe of the group
    group_df = group_df.reset_index().drop('index',1)
    
    # get start and end of dive
    stime = group_df['Time'].iloc[0]
    etime = group_df['Time'].iloc[-1]
    
    # ignore dives shorter than 1 minute (at_depth)
    if (etime-stime) < dive_duration_thresh:
        print(etime-stime)
        continue
    
    # only take last minute 
    group_df = group_df.iloc[-h:]
    
    # time
    time.append(group_df['Time'])
    
    # shaplet features
    depth.append(group_df['elevation'])
    roll.append(np.abs(group_df['roll']))
    
    # scalar features
    htv.append(group_df['d_head'].sum())
    hv.append(circvar(group_df['head'],nan_policy='omit'))
    jp.append(group_df['jerk'].max())
    
    # get the label
    label = False
    for event in forage_events:
        label = (label) or ((stime <= event) and (etime >= event))
    labels.append(label)

    
    if label:
        plt.plot(time[-1],depth[-1])
        plt.show()
        plt.plot(time[-1],roll[-1])
        plt.show()
        
        print(htv[-1])
        print(hv[-1])
        print(jp[-1])

In [ ]:
# set parameters
surface_thresh = 20
dive_duration_thresh = datetime.timedelta(minutes=60)
between_dive_thresh = datetime.timedelta(seconds=2)
h = 3000

# divide dataframe into segments
df_downsampled = df[['Time','depth']].iloc[::h]
df_downsampled['segnum'] = range(len(df_downsampled))

# check which segments start at depth
df_downsampled['at_depth'] = df_downsampled['depth'] >= surface_thresh

# label the dataframe by foraging event
df_downsampled['label'] = False
for time in forage_events:
    df_downsampled['label'] = (df_downsampled['label']) | \
                              ((df_downsampled['Time'] <= time) & \
                              (df_downsampled['Time'] >= time-datetime.timedelta(seconds=h/50)))

# combine segments with full dataframe
df = df.merge(df_downsampled,how='left')
df[['segnum','at_depth','label']] = df[['segnum','at_depth','label']].fillna(method='ffill')

# get rid of segments above threshold
df = df[df['at_depth']].drop('at_depth',axis='columns')

# Format the data for inference

In [ ]:
gbo = df.groupby('segnum')

time = []
depth = []
pitch = []
roll = []
head = []
labels = []

for segnum,group_df in gbo:
    
    group_df = group_df.reset_index().drop('index',1)
    
    time.append(group_df['Time'])
    depth.append(group_df['elevation'])
    pitch.append(group_df['pitch'])
    roll.append(np.abs(group_df['roll']))
    head.append((group_df['head']-group_df['head'][0]+np.pi)%(2*np.pi)-np.pi)
    
    labels.append(group_df.label.iloc[0])
    
    if (group_df.label.iloc[0]) | (segnum < 10):
        print(group_df.label.iloc[0])
        print('')
        print(group_df['d_head'].mean())
        print(circvar(group_df['head'],nan_policy='omit'))
        #print(group_df['jerk'].max())
        group_df.plot(x='Time',y='elevation')
        #group_df.plot(x='Time',y=['roll'])
        group_df.plot(x='Time',y=['head','d_head'])
        #group_df.plot(x='Time',y='VeDBA')
        #group_df.plot(x='Time',y='jerk')
        plt.show()

In [ ]:
a(group_df['head'][:1000])

In [ ]:
plt.plot(group_df['head'][1000:])

In [ ]:
train_x = pd.DataFrame({'Time':time,
                        'depth':depth,
                        'pitch':pitch,
                        'roll':roll,
                        'head':head})
train_y = np.array(labels)

# Train the model

In [ ]:
num_negatives = 50

inds = np.random.choice(len(train_x),num_negatives,replace=False)
inds = np.union1d(inds,np.where(train_y)[0])

min_shapelet_length = 25 # 0.5 seconds
max_shapelet_length = 250 # 5 seconds  

# How long (in minutes) to extract shapelets for.
# This is a simple lower-bound initially;
# once time is up, no further shapelets will be assessed
time_contract_in_mins = 30

# The initial number of shapelet candidates to assess per training series.
# If all series are visited and time remains on the contract then another
# pass of the data will occur
initial_num_shapelets_per_case = 10

# Whether or not to print on-going information about shapelet extraction.
# Useful for demo/debugging
verbose = 2

sts = []
st_dfs = []

for col_num,col in enumerate(train_x.columns[1:]):
    
    print('')
    print(col)
    print('')

    st = ContractedShapeletTransform(
        time_contract_in_mins=time_contract_in_mins,
        num_candidates_to_sample_per_case=initial_num_shapelets_per_case,
        min_shapelet_length = min_shapelet_length,
        max_shapelet_length = max_shapelet_length
        verbose=verbose,
    )

    st.fit(train_x.iloc[inds][[col]], train_y[inds])
    sts.append(st)
    
    st_dfs.append(st.transform(train_x.iloc[inds][[col]]))

In [ ]:
# plot data from this transform
train_x0 = train_x.iloc[inds]

for col_num in range(len(train_x.columns)-1):
    
    print(train_x.columns[col_num+1])
    print('')

    # for each extracted shapelet (in descending order of quality/information gain)
    for s in sts[col_num].shapelets[0:4]:

        # summary info about the shapelet
        print(s)

        # plot the series that the shapelet was extracted from
        plt.plot(train_x0.iloc[s.series_id, col_num+1], "gray")

        # overlay the shapelet onto the full series
        plt.plot(
            list(range(s.start_pos, (s.start_pos + s.length))),
            train_x0.iloc[s.series_id, col_num+1][s.start_pos : s.start_pos + s.length],
            "r",
            linewidth=3.0,
        )
        plt.show()
        
    plt.scatter(st_dfs[col_num][0],st_dfs[col_num][1],c=train_y[inds])
    plt.show()

In [ ]:
(np.diff(train_x.iloc[index]['roll']) + np.pi)

In [ ]:
((np.diff(train_x.iloc[index]['roll']) + np.pi) % (2*np.pi)) - np.pi

In [ ]:
train_x

In [ ]:
train_x.iloc[index]['roll'].rolling(5,center=True).mean().diff()

In [ ]:
for index in [5,6,7,37,43,135,139,153]:

    plt.plot(train_x.iloc[index]['Time'],train_x.iloc[index]['depth'])
    plt.show()
    #plt.plot(train_x.iloc[index]['Time'],train_x.iloc[index]['pitch'])
    plt.plot(train_x.iloc[index]['Time'],train_x.iloc[index]['roll'])
    plt.plot(train_x.iloc[index]['Time'],np.abs(((train_x.iloc[index]['roll'].diff() + np.pi) % (2*np.pi)) - np.pi))
    #plt.plot(train_x.iloc[index]['Time'],train_x.iloc[index]['head'])
    #plt.legend(['pitch','roll','depth'])
    plt.show()
    print(np.mean(np.abs(((train_x.iloc[index]['roll'].rolling(11,center=True).mean().diff(11) + np.pi) % (2*np.pi)) - np.pi)))
    print(labels[index])

In [ ]:
for col_num in range(4):
    print(train_x.columns[col_num+1])
    print('')
    plt.scatter(st_dfs[col_num][0],st_dfs[col_num][1],c=train_y[inds])
    plt.show()
    plt.scatter(st_dfs[col_num][2],st_dfs[col_num][3],c=train_y[inds])
    plt.show()

In [ ]:
# Saving the objects:
with open('objs.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([sts,st_dfs], f)

In [ ]:
# Getting back the objects:
with open('objs.pkl') as f:  # Python 3: open(..., 'rb')
    sts,st_dfs = pickle.load(f)